# Plots

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
import numpy as np
import glob

## Plot different classes in different colors on the same image

In [2]:
# select slice and mode 
# slice ranging from 400th to 800th, 401 in total
# seg_model: k-means/gmm/dragonfly
# mode: number of classes, either 16, 32, 64, 128 or 4 for dragonfly
slice_num = 400
seg_model = 'k-means'
seg_nd = '3d'
mode = 16

In [3]:
assert (seg_model == 'dragonfly') == (mode == 4)

In [6]:
# data folder path
current_path = os.getcwd()

# dragonfly
def get_dragonfly_images(slice_num):
    s = slice_num - 400 + 1
    s = str(s).zfill(3)
    dragonfly_bassanite = os.path.join(current_path, 'VA10_0050_Bassanite\VA10_0050_Bassanite{}.tiff'.format(s))
    dragonfly_celestite = os.path.join(current_path, 'VA10_0050_Celestite\VA10_0050_Celestite{}.tiff'.format(s))
    dragonfly_gypsum = os.path.join(current_path, 'VA10_0050_Gypsum\VA10_0050_Gypsum{}.tiff'.format(s))
    dragonfly_pore = os.path.join(current_path, 'VA10_0050_Pores\VA10_0050_Pores{}.tiff'.format(s))

    bassanite = cv2.imread(dragonfly_bassanite)
    celestite = cv2.imread(dragonfly_celestite)
    gypsum = cv2.imread(dragonfly_gypsum)
    pore = cv2.imread(dragonfly_pore)

    return bassanite, celestite, gypsum, pore

def get_dragonfly_grayscale(slice_num):
    s = slice_num - 400 + 1
    s = str(s).zfill(3)
    dragonfly_bassanite = os.path.join(current_path, 'VA10_0050_Bassanite\VA10_0050_Bassanite{}.tiff'.format(s))
    dragonfly_celestite = os.path.join(current_path, 'VA10_0050_Celestite\VA10_0050_Celestite{}.tiff'.format(s))
    dragonfly_gypsum = os.path.join(current_path, 'VA10_0050_Gypsum\VA10_0050_Gypsum{}.tiff'.format(s))
    dragonfly_pore = os.path.join(current_path, 'VA10_0050_Pores\VA10_0050_Pores{}.tiff'.format(s))

    bassanite = cv2.imread(dragonfly_bassanite, cv2.IMREAD_GRAYSCALE) / 255
    celestite = cv2.imread(dragonfly_celestite, cv2.IMREAD_GRAYSCALE) / 255
    gypsum = cv2.imread(dragonfly_gypsum, cv2.IMREAD_GRAYSCALE) / 255
    pore = cv2.imread(dragonfly_pore, cv2.IMREAD_GRAYSCALE) / 255

    return bassanite, celestite, gypsum, pore

# gmm / k-means
def get_seg_image_path(model, nd, mode, slice_num):
   seg_list = glob.glob(os.path.join(current_path, 'large_clusters_rec', model,  nd, 'cluster_{}'.format(mode), slice_num))



In [5]:
bassanite, celestite, gypsum, pore = get_dragonfly_images(slice_num)


In [4]:
# Color Palette
black = [0,0,0]
white = [255,255,255]
color_1 = [118,42,131]
color_2 = [175,141,195]
color_3 = [231,212,232]
color_4 = [217,240,211]
color_5 = [127,191,123]
color_6 = [27,120,55]

In [5]:
# load related images
#if mode == 4:
bassanite, celestite, gypsum, pore = get_dragonfly_images(471)
bassanite[np.all(bassanite == white, axis=-1)] = color_2
celestite[np.all(celestite == white, axis=-1)] = color_3
gypsum[np.all(gypsum == white, axis=-1)] = color_4
pore[np.all(pore == white, axis=-1)] = color_5

img = bassanite + celestite + gypsum + pore
cv2.imwrite('dragonfly_visualisation_report_471.png', img)

True

In [17]:
# test intersection of classes from dragonfly => no overlapping
b_gray, c_gray, g_gray, p_gray = get_dragonfly_grayscale(400)
all_add = b_gray + c_gray + g_gray + p_gray
all_add

array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.]])

In [20]:
print((all_add == 0).sum())   # 25492 pixels were not classified as any classes here

25492


In [21]:
25492 / (700*855)   # indeed within the 95% accuaracy range

0.04259314954051796

# labeling segmentation results and combine same class together

pore: 0

gypsum: 1

celestite: 2

bassanite: 3

In [10]:
# for my segmentation results: concate up for 
# since the clusters are gathering because of the same centroids, if there is an empty cluster, then that means in that image, there isn't any pixel that could be considered as that cluster.
# Empty is a signal of no some class, so it is important to keep it as that class. 


"""need to find the average of each cluster, because as z goes larger, gypsum might goes down, pores and bassanie might grow and hence has more dominant data at around 750-800 slices but are tie with gypsum at the beginning.  """

# the following code was for k-means 3d 16

label = [0]*16
label[0] = 1
label[1] = 2
label[2] = 0  # 0/1 weird that presicion of pore was obviously larger but then it drops
label[3] = 0
label[4] = 1
label[5] = 3
label[6] = 0
label[7] = 1
label[8] = 1
label[9] = 3
label[10] = 0
label[11] = 3
label[12] = 0  #0/1  # weird that presicion of pore was obviously larger but then it drops
label[13] = 1  #0/0/3
label[14] = 1
label[15] = 3



In [11]:
seg_model = 'k-means'
seg_nd = '3d'
mode = 16


df = pd.read_csv('{}_{}_{}_f1.csv'.format(seg_model, seg_nd, mode))

In [12]:
same_class = np.array(label*401)  # the res from same group mush have same label

df['same_class'] = same_class
df

,slice,current_cluster,pore_micro_precision,pore_micro_recall,pore_micro_f1,pore_macro_precision,pore_macro_recall,pore_macro_f1,gypsum_micro_precision,gypsum_micro_recall,...,celestite_macro_precision,celestite_macro_recall,celestite_macro_f1,bassanite_micro_precision,bassanite_micro_recall,bassanite_micro_f1,bassanite_macro_precision,bassanite_macro_recall,bassanite_macro_f1,same_class
0,400,0,0.012649,0.029551,0.017715,0.012855,0.030710,0.017283,0.918107,0.162058,...,0.000000,0.000000,0.000000,0.050779,0.036827,0.042692,0.051571,0.036910,0.041451,1
1,400,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2
2,400,2,0.495717,0.185133,0.269586,0.485919,0.192986,0.266431,0.445632,0.012575,...,0.000000,0.000000,0.000000,0.040623,0.004710,0.008441,0.042345,0.005007,0.008756,0
3,400,3,0.773465,0.166946,0.274618,0.756065,0.163197,0.256818,0.129570,0.002113,...,0.000000,0.000000,0.000000,0.089485,0.005996,0.011239,0.082403,0.005678,0.010376,0
4,400,4,0.048145,0.068911,0.056686,0.048326,0.074037,0.054871,0.881532,0.095334,...,0.000000,0.000000,0.000000,0.054871,0.024381,0.033761,0.055902,0.025334,0.033623,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6411,800,11,0.003934,0.002790,0.003265,0.004146,0.002722,0.003128,0.000000,0.000000,...,0.000179,0.005848,0.000346,0.990529,0.268365,0.422312,0.990973,0.267241,0.413764,3
6412,800,12,0.360981,0.117477,0.177265,0.348757,0.115484,0.171463,0.328915,0.009191,...,0.000000,0.000000,0.000000,0.281213,0.034968,0.062202,0.284500,0.034887,0.061250,0
6413,800,13,0.224580,0.054941,0.088285,0.231613,0.055754,0.085039,0.428888,0.009009,...,0.000000,0.000000,0.000000,0.321297,0.030033,0.054932,0.309045,0.030372,0.054245,1
6414,800,14,0.006191,0.012941,0.008375,0.006166,0.014447,0.008237,0.942736,0.169204,...,0.000000,0.000000,0.000000,0.037221,0.029727,0.033055,0.038444,0.029277,0.032186,1


In [1]:
zero = df.loc[df['current_cluster'] == 0]

NameError: name 'df' is not defined

In [2]:
zero.describe()

NameError: name 'zero' is not defined

# Notes

In [ ]:
# 3d and 4d similarity, strong active sense in the middle. Even though the representation of activeness in term of time and space might be different, but they are both presenting the same reaction process. Some other regions at two ends are experiencing the previous stage of the middle regions. 
# Similarly, some later reaction stage would happen at the region at the ends.
# Hence, understanding z axis is important for understanding the axis of time. 



# Image comparison

In [32]:
p1 = os.path.join(os.getcwd(), 'report_images\T0050_0800_4d_cluster_6_2.png')
p2 = os.path.join(os.getcwd(), 'report_images\T0050_0800_4d_cluster_6_1.png')
img1 = ~cv2.imread(p1, 0)
img2 = ~cv2.imread(p2, 0)

both = img1 + img2
flip = ~both
cv2.imwrite('report_images/T0050_0800_4d_cluster_6_1+2.png', flip)

# cv2.imshow('d', both)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


# diff = (img1 != img2).any(axis=2).sum()
# ratio = 1- (diff / (700*855))
# print(diff)
# print(ratio)


True

In [20]:

# both = x+y
# invert_both = ~both
# #cv2.imshow('d', invert_both)
# cv2.imshow('d', x)
# cv2.waitKey(0)

# cv2.destroyAllWindows()
check = x+y


In [21]:
check

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [27]:
# compare if 6-2 plus 6-1 and 3-2
p3 = os.path.join(os.getcwd(), 'report_images\T0050_0800_4d_cluster_3_2.png')
img3 = cv2.imread(p3, 0)
im3 = img3.flatten()
z = ~im3


In [30]:
dif = (check != z).sum()
dif/(700*855)

0.149593984962406

In [28]:
cv2.imshow('w', x)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Images for Report

In [4]:
file_path = os.path.join(os.getcwd(), 'VA10_Pc200_Ram25_Pf50_T125_0002\VA10_Pc200_Ram25_Pf50_T125_0002_0001.rec.8bit.tif')
img = cv2.imread(file_path, -1) #[353:1053, 282:1137]
cv2.imwrite('report_images/raw_data_to_png/VA10_Pc200_Ram25_Pf50_T125_0002_0001.rec.8bit..png', img)


True

In [7]:
# color_1 = [118,42,131]
# color_2 = [175,141,195]
# color_3 = [231,212,232]
# color_4 = [217,240,211]
# color_5 = [127,191,123]
# color_6 = [27,120,55]
black = [0,0,0]
gray = [150,150,150]
white = [255,255,255]
orange = [92,204,254]
red = [28,26,227]
pink = [185,180,251]
color_2 = [180,218,161]
color_3 = [196,182,65]
color_4 = [204,255,255]
color_5 = [168,94,34]
blue = [184,127,44]

In [8]:

bassanite, celestite, gypsum, pore = get_dragonfly_images(618)
bassanite[np.all(bassanite == white, axis=-1)] = orange
celestite[np.all(celestite == white, axis=-1)] = blue
gypsum[np.all(gypsum == white, axis=-1)] = gray
pore[np.all(pore == white, axis=-1)] = red

img = bassanite + celestite + gypsum + pore
cv2.imwrite('report_images\dragonfly_visualisation_report_618.png', img)

True

In [22]:
# gmm / k-means
def color_unsupervised_pred_by_slice(manual_label, seg_model, seg_nd, cluster_num, slice_num):
    current_path = os.getcwd()
    seg_path = os.path.join(current_path, 'large_clusters_rec', seg_model, seg_nd, 'cluster_{}'.format(cluster_num))
    seg_res_list = glob.glob(os.path.join(seg_path, str(slice_num), '*.png'))
    
    img = 0

    assert len(manual_label) == len(seg_res_list) == cluster_num

    for seg in seg_res_list:
        # our segmentation results has invered color, black [0,0,0] is the interested class
        c = int(os.path.basename(seg[-16:-13]))
        if manual_label[c] == 1:  # pore
            pore = cv2.imread(seg)
            pore[np.all(pore == black, axis=-1)] = red
            pore[np.all(pore == white, axis=-1)] = black
            img += pore
        elif manual_label[c] == 2:  #gypsum
            gypsum = cv2.imread(seg)
            gypsum[np.all(gypsum == black, axis=-1)] = gray
            gypsum[np.all(gypsum == white, axis=-1)] = black
            img += gypsum
        elif manual_label[c] == 3:  #celestite
            celestite = cv2.imread(seg)
            celestite[np.all(celestite == black, axis=-1)] = blue
            celestite[np.all(celestite == white, axis=-1)] = black
            img += celestite
        elif manual_label[c] == 4:  #bassanite
            bassanite = cv2.imread(seg)
            bassanite[np.all(bassanite == black, axis=-1)] = orange
            bassanite[np.all(bassanite == white, axis=-1)] = black
            img += bassanite

    return img

In [23]:
def auto_label_last_201(seg_model, seg_nd, cluster_num, mode, threshold=0.5):
    # cluster_num: the total number of clusters
    assert mode in [1,2,3,4], "Invalid mode: mode should be integer in [1,2,3,4]."
    if mode == 1:
        assert threshold == 0.5, "Mode 1 requires threshold = 0.5."

    csv_file = os.path.join(os.getcwd(), 'evaluation_rec_f1', '{}_{}_{}_f1.csv'.format(seg_model, seg_nd, cluster_num))
    df = pd.read_csv(csv_file, usecols=['slice', 'current_cluster','pore_micro_precision', 'pore_micro_f1', 'gypsum_micro_precision', 'gypsum_micro_f1', 'celestite_micro_precision', 'celestite_micro_f1', 'bassanite_micro_precision', 'bassanite_micro_f1'])
    df =  df.loc[df['slice'] >= 600]  # only use training set

    label = [0]*cluster_num

    for i in range(cluster_num):
        one_cluster = df.loc[df['current_cluster'] == i]
        stats = one_cluster.mean()
        precisions = [stats[2], stats[4], stats[6], stats[8]]
        fscores = [stats[3], stats[5], stats[7], stats[9]]
        p_max = max(precisions)
        p_idx = np.argmax(precisions)
        f_idx = np.argmax(fscores)
        if mode in [1,2]:
            if p_max <= threshold:
                idx = f_idx
            else:
                idx = p_idx
        elif mode == 3:
            idx = f_idx
        else:
            idx = p_idx
            
        class_num = idx + 1
        label[i] = class_num

    return label

In [12]:
from auto_label import PRECISION_SOLO

In [28]:
seg_model = 'k-means'
seg_nd = '4d'
cluster_num = 128
mode = PRECISION_SOLO
threshold = 0

In [29]:
label = auto_label_last_201(seg_model, seg_nd, cluster_num, mode, threshold)

In [15]:
len(label)

128

In [30]:
img = color_unsupervised_pred_by_slice(label, seg_model, seg_nd, cluster_num, slice_num=471)

In [31]:
cv2.imwrite('report_images\k-means_4d_128_report_471.png', img)

True

#### Transfer tif to png

In [4]:
p1 = os.path.join(os.getcwd(), 'VA10_Pc200_Ram25_Pf50_T125_0140\VA10_Pc200_Ram25_Pf50_T125_0140_0001.rec.8bit.tif')
img1 = cv2.imread(p1, -1)

cv2.imwrite('report_images/raw_data_to_png/VA10_Pc200_Ram25_Pf50_T125_0140_0001.rec.8bit.png', img1)

True